# MAD Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
22/03/2024

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit



In [3]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x0000027B0E0A71B0>
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [4]:
# SENSIBILITE CAPTEUR
S_force =  39.2  # mV/N
G_force = 100

# cannal d'acquisition
Cannal = 1

In [5]:
dossier = 'mes_2024.03.22'
nom_anche = '/A'
nbr_anche = 3
seuil = 150

## Fonctions Modélisations

In [6]:
# Caracteristique non lineaire S(p)
# p : Pression dans la bouche
# S00 : Ouverture a p=0
# pM : Pression de plaquage
# Coude : Etendue de la partie coude en Pa
# Fuite : Ouverture equivalente a la fuite


def caracNL_BG(p, S00, pM, Pb, SL):
    p = p - min(p)
    # S00 = param[0]
    # pM = param[1]
    # Pb = param[2]
    # SL = param[3]

    # Calcul de la pente de la droite linéaire
    C = S00 / pM
    PprimeM = (S00 - SL) / C
    A = (S00 - SL) / (4 * Pb * PprimeM)

    pb1 = PprimeM - Pb  # pression début coude
    pb2 = PprimeM + Pb  # pression fin coude

    # Création des indices pour différentes parties de la courbe
    ind1 = np.argwhere(p <= pb1)  # partie linéaire
    ind2 = np.argwhere((p > pb1) & (p < pb2))  # coude
    ind3 = np.argwhere(p >= pb2)  # fuite

    # Initialisation de la section S
    S = np.zeros_like(p)
    # print(S.shape)

    # Calcul des valeurs de S pour les différentes parties de la courbe
    S[ind1] = S00 - C * p[ind1]  # partie linéaire avant enroulement
    S[ind2] = A * (p[ind2] - PprimeM - Pb) ** 2 + SL  # coude -> équation du second degré
    S[ind3] = SL  # fuite
    # print('S = ', S)
    return S


In [7]:
# force : pression généralisée (= force de la lèvre )
# S : section d'ouverture

# S00 : section d'ouverture à force nulle (ordonnée à l'origine)
# PM: pression de plaquage
# Coude : Pb dans le modèle
# Sfuites : section de fuites
# Paramopt : parametres optimaux [S00, PM, Coude, Sfuites]
# Smodelepg: section théorique correspondant aux parametres optimaux

def identification_modele(force, S):
    # Identification grossière des paramètres pour l'initialisation de l'optimisation
    Smax = max(S)
    indmax = np.where(S > 0.5 * Smax)
    
    # partie linéaire au début de la caractéristique
    P = np.polyfit(force[indmax], S[indmax], 1)
    Cest = -P[0]  # pente
    S0est = P[1]  # ordonnée à l'origine
    Stheo = np.polyval(P, force[indmax])

    #####################################################################
    # Paramètres initiaux de l'optimisation
    PM0 = S0est / Cest # solution de l'équation de la droite de la partie linéraire (abscisse de Surface = 0)
    Sfuites0 = 0 
    
    # Valeur initiale de Coude0 variable pour trouver l'erreur min du modèle
    # Attention ne pas utiliser Coude0 trop grand sans quoi il y a convergence vers Coude = PM !
    Coude0 = PM0 / 4
    
    
    p0 = [S0est, PM0, Coude0, Sfuites0]
    Paramopt = curve_fit(caracNL_BG, xdata=force, ydata=S, p0=p0 ,method='lm') # Use non-linear least squares to fit a function, f, to data
    # Smodelepg = caracNL_BG(force, *Paramopt)
    # erreuropt = np.sqrt(np.sum((Smodelepg - S) ** 2) / np.sum(S ** 2)) * 100
    
    Paramopt = Paramopt[0]
    print(Paramopt)
    print(type(Paramopt))
    surface = Paramopt[0]
    pression = Paramopt[1]
    coude = Paramopt[2]
    fuite = Paramopt[3]
    # Tracé des résultats
    pgt = np.linspace(0, 13, 300)
    ST = caracNL_BG(pgt, surface,pression, coude, fuite)
    
    plt.plot(force, S, '-+', lw=2, label='Données expérimentales')
    plt.plot(pgt, ST, 'k', lw=2, label='Modèle ajusté')
    
    # Affichage des traits de coustruction du modèle 
    # plt.plot(0, Paramopt[0], '+r', lw=6)
    # plt.plot(Paramopt[1], 0, '+r', lw=6)
    # plt.plot([Paramopt[1], 0], [0, Paramopt[0]], '-.r', lw=1)
    # plt.plot(Paramopt[1] + Paramopt[2], 0, '+g', lw=6)
    # plt.plot(Paramopt[1] - Paramopt[2], 0, '+g', lw=6)
    # plt.plot(0, Paramopt[3], '+b', lw=6)
    # plt.plot([0, 6], [Paramopt[3], Paramopt[3]], ':b', lw=1)
    # limit = np.linspace(0, 14, 100)
    # plt.plot((Paramopt[1] + Paramopt[2]) * np.ones_like(limit), limit, '--g', lw =1)
    # plt.plot((Paramopt[1] - Paramopt[2]) * np.ones_like(limit), limit, '--g', lw =1)
    
    plt.grid()
    plt.xlabel('Pression généralisée (force)')
    plt.ylabel('Section d\'ouverture (S)')
    plt.legend()
    plt.show()

    return Paramopt

# Applications

In [8]:
plt.close('all')
force = np.loadtxt(dossier + '/force_A2.txt')
offset = min(force)
force = (force - offset)/S_force
surf_canal = np.loadtxt(dossier + '/air_canal_A2.txt')


Paramopt = identification_modele(force, surf_canal)

raideur = Paramopt[1]/Paramopt[0]
# print(raideur)


[12.80833902  3.53455068  1.99390893  0.04739498]
<class 'numpy.ndarray'>


In [11]:
# Modélisation par simple polyfit
polyfit_model = False

if polyfit_model : 
    plt.close('all')

    coeffs = np.polyfit(force, surf_canal, 3)
    poly_fit = np.poly1d(coeffs)
    print(coeffs)

    # Affichage des résultats
    plt.scatter(force, surf_canal, label='Data')
    plt.plot(force, poly_fit(force), label='Polynomial fit', color='red')
    plt.xlabel('Force')
    plt.ylabel('Surface')
    plt.legend(loc='best')
    plt.show()